In [2]:
!pip install dash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 KB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.1/133.1 KB 26.7 MB/s eta 0:00:00
  Attempting uninstall: MarkupSafe
    Found existing installation: MarkupSafe 2.0.0
    Not uninstalling markupsafe at /shared-libs/python3.9/py-core/lib/python3.9/site-packages, outside environment /root/venv
    Can't uninstall 'MarkupSafe'. No files were found to uninstall.
  Attempting uninstall: Jinja2
    Found existing installation: Jinja2 2.11.3
    Not uninstalling jinja2 at /shared-libs/python3.9/py-core/lib/python3.9/site-packages, outside environment /root/venv
    Can't uninstall 'Jinja2'. No files were found to uninstall.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [5]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                            options=[
                                {'label': 'All Sites', 'value': 'ALL'},
                                {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                {'label':'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                {'label':'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                {'label':'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
                            ],
                            value='ALL',
                            placeholder="Select a Launch Site here",
                            searchable=True
                            ),

                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider', min=0, max=10000,
                                step=1000, marks={0: '0', 2500: '2500', 5000: '5000', 7500: '7500', 10000:'10000'}, 
                                value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value')   
            )
def get_pie_chart(value): 
    if value == 'ALL':
        filtered_df = spacex_df.groupby('Launch Site')['class'].mean().reset_index()
        fig = px.pie(filtered_df, values='class', 
        names='Launch Site', 
        title='Total Success Launch All Sites')
        return fig
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == value]
        values = filtered_df['class'].value_counts()
        labels = ['Class 1', 'Class 0']
        fig = px.pie(values=values, names=labels, title=f'Percentage of Classes for {value}')
        return fig
        # return the outcomes piechart for a selected site
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'),
              Input(component_id="payload-slider", component_property="value")
              )
def get_range_bar(value,value_pay): 
    if value == 'ALL':
        filtered_df=spacex_df[(spacex_df['Payload Mass (kg)']>=value_pay[0]) & (spacex_df['Payload Mass (kg)']<=value_pay[1])]
        fig1 = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', 
        color = 'Booster Version Category', symbol='Booster Version Category', 
        title='Payload vs. Outcome for {}'.format(value))
        fig1.update_traces(marker_size=10)

        return fig1
    else:
        df1=spacex_df[spacex_df["Launch Site"]==value]
        filtered_df=df1[(df1['Payload Mass (kg)']>=value_pay[0]) & (df1['Payload Mass (kg)']<=value_pay[1])]
        fig1 = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', 
        color = 'Booster Version Category', symbol='Booster Version Category',
        title='Payload vs. Outcome for {}'.format(value))
        fig1.update_traces(marker_size=10)

        return fig1
# Run the app
if __name__ == '__main__':
    app.run_server()




Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off
 * Running on http://127.0.0.1:8050
Press CTRL+C to quit


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=285357d6-abae-4dcd-b938-2e24042d8942' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>